In [28]:
from IsingLib import Model, Hamil, DaSampler, TmpDaSampler
from pyqubo import Array, Sum, Constraint
import numpy as np

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

np.set_printoptions(threshold=30)
np.set_printoptions(edgeitems=20)
np.set_printoptions(linewidth=200)

In [29]:
city = 5
vartype = 'BINARY'
x = Array.create('x', (city, city), vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i+1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
distance = distance + distance.T
time_const = Constraint(Sum(0, city, lambda i: (Sum(0, city, lambda j: x[i, j]) - 1) ** 2), label='time')
city_const = Constraint(Sum(0, city, lambda j: (Sum(0, city, lambda i: x[i, j]) - 1) ** 2), label='city')

In [30]:
const_model = Model.create(vartype, time_const+city_const)
aim_model = Model(vartype, distance, symbol='x', shape=(city, city))

In [31]:
h = Hamil(aim=aim_model, const=const_model)

In [32]:
# print(h)

In [33]:
model = h.compile(const=10)

In [34]:
ans = model.solve('sa',
                  temperature_start=1000000.0,
                  temperature_decay=0.9998
)

In [35]:
print(ans.states)
print(ans.energies)

[{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 1, 'x[0][3]': 0, 'x[0][4]': 0, 'x[1][0]': 1, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 1, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 0, 'x[3][3]': 1, 'x[3][4]': 0, 'x[4][0]': 0, 'x[4][1]': 1, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0}]
[35.0]


In [36]:
for state in ans.states:
    print(h.check_constraints(state, 'const'))

0.0


In [49]:
sampler = TmpDaSampler(api_key='../../../da_key_2.json', version=2)
da_model = model.to_da()
response_da = sampler.sample(**da_model)

In [50]:
print(response_da.states[0])

{'x[0][0]': 0, 'x[0][1]': 0, 'x[2][0]': 0, 'x[2][1]': 1, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 1, 'x[3][3]': 0, 'x[3][4]': 0, 'x[0][2]': 0, 'x[4][0]': 1, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 1, 'x[1][4]': 0}


In [51]:
sampler = EmbeddingComposite(DWaveSampler())
bqm = (model).to_bqm()
response_dwave = sampler.sample(bqm)

In [52]:
print(response_dwave.first.sample)

{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 0, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 1, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0}
